In [1]:
import pandas as pd

In [2]:
# CITY = 'beijing'
CITY = 'shanghai'

In [3]:
room_df = pd.read_csv('./data/{}/room.csv.bak'.format(CITY))
host_df = pd.read_csv('./data/{}/host.csv.bak'.format(CITY))

In [4]:
def filter_noise(raw_room_df:pd.DataFrame):
    room_df = raw_room_df.copy(deep=True)
    room_df = room_df.drop(['amenities'], axis=1)
    drop_index_list = room_df[room_df['price']==0].index.tolist()
    room_df = room_df.drop(drop_index_list)
    # three sigma principle
    neighbourhood_group = room_df.groupby('neighbourhood')
    mean_price = neighbourhood_group['price'].mean() 
    std_price = neighbourhood_group['price'].std()
    up_lim = mean_price +  3 * std_price
    low_lim = mean_price - 3* std_price
    valid_room_df_list = []
    for name, room in neighbourhood_group:
        valid_room_df_list.append(room_df[(room_df['neighbourhood']==name) & (room_df['price'] <= up_lim[name]) & (room_df['price'] >= low_lim[name])])
    room_df = pd.concat(valid_room_df_list)
    return room_df

In [5]:
def format_room(raw_room_df:pd.DataFrame):
    room_df = raw_room_df.copy(deep=True)
    room_df = room_df.fillna(value=0)
    # fill blanks before chang column type
    room_df['bedrooms']= room_df['bedrooms'].astype(int)
    room_df['beds']=room_df['beds'].astype(int)
    return room_df

In [6]:
def get_best_room_df(raw_room_df:pd.DataFrame):
    room_df = raw_room_df.copy(deep=False)
    review_num_lim = room_df['number_of_reviews'].quantile(0.9)
    review_score_lim = room_df['review_scores_rating'].quantile(0.9)
    best_room_df = room_df[(room_df['number_of_reviews']>=review_num_lim) & (room_df['review_scores_rating']>=review_score_lim)]
    return best_room_df    

In [7]:
def preprocess_room(raw_room_df:pd.DataFrame):
    room_df = raw_room_df.copy(deep=True)
    room_df = filter_noise(room_df)
    room_df = format_room(room_df)
    best_room_df = get_best_room_df(room_df)
    return room_df, best_room_df

In [8]:
room_df, best_room_df = preprocess_room(room_df)

In [9]:
room_df.to_csv('./data/{}/room.csv'.format(CITY),index=False)

In [10]:
best_room_df.to_csv('./data/{}/best_room.csv'.format(CITY),index=False)

In [11]:
room_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14676 entries, 76 to 14874
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   room_id               14676 non-null  int64  
 1   host_id               14676 non-null  int64  
 2   neighbourhood         14676 non-null  object 
 3   name                  14676 non-null  object 
 4   latitude              14676 non-null  float64
 5   longitude             14676 non-null  float64
 6   room_type             14676 non-null  object 
 7   accommodates          14676 non-null  int64  
 8   bedrooms              14676 non-null  int32  
 9   beds                  14676 non-null  int32  
 10  instant_bookable      14676 non-null  object 
 11  first_review          14676 non-null  object 
 12  last_review           14676 non-null  object 
 13  number_of_reviews     14676 non-null  int64  
 14  review_scores_rating  14676 non-null  float64
 15  price             